##### Creation Date:
##### February 13 2022
##### Created By Alperen KOLAMUC

| String Etiketler |
|------------------|
| Kraliçe          |
| Kral             |
| Prenses          |
| Prens            |
| Köylü Kız        |
| Köylü Çocuk      |

* Bu Etiketlerin hepsi `String` bilgisayar bunları nasıl görecek? Çünkü bilgisayar sadece sayılarla işlem yapıyordu. Oysaki `"Kraliçe"` = 0x4B 0x72 0x61 0x6C 0x69 0xE7 0x65 ASCII kodlarından oluşuyor. O zaman ben `String` ifadelerime unique sayılar verip onlarla ifade edebilirim.

E.g:

| String Etiketler |   |
|:----------------:|:-:|
|      Kraliçe     | 6 |
|       Kral       | 3 |
|      Prenses     | 5 |
|       Prens      | 2 |
|     Köylü Kız    | 4 |
|    Köylü Çocuk   | 1 |

* Evet buraya kadar `String` etiketleri bilgisayara tanıtmayı başardık fakat burada şöyle bir sıkıntı ortaya çıkıyor. `"BENZERLİK"`. Köylü Çocuk = 1, Prens = 2. Değerleri bir birine yakın bu ikisi benzer midir? Tabiki değildir. O zaman `Label Encoding` yaparken sayıları kafamıza göre girmememiz lazım.

| String Etiketler |   |
|:----------------:|:-:|
|      Kraliçe     | 2 |
|       Kral       | 1 |
|      Prenses     | 4 |
|       Prens      | 3 |
|     Köylü Kız    | 6 |
|    Köylü Çocuk   | 5 |

* Şimdi tablo daha anlamlı hale geldi.
* Peki tabloyu yukarıda `Soyluluk` kavramına göre düzenledik. Peki ben aynı zamanda `Cinsiyet`'e göre de numaralandırmak istesem nasıl olacaktı? O zaman `Cinsiyet` ve `Soyluluk` durumlarına göre numaralandıracağız.

| Code | **Cinsiyet** | **Soyluluk** |
|------|--------------|--------------|
|   0  | Erkek        | Soysuz       |
|   1  | Kadın        | Soylu        |
|   2  |              | Çok Soylu    |

* Bu yeni bilgilere göre tablomuz artık;

| String Etiketler | (Cinsiyet,Soy)|
|:----------------:|:-----:|
|      Kraliçe     | (1,2) |
|       Kral       | (0,2) |
|      Prenses     | (1,1) |
|       Prens      | (0,1) |
|     Köylü Kız    | (1,0) |
|    Köylü Çocuk   | (0,0) |


* Böylece artık vektörel olarakta objeler arasında ilişki kurabiliriz


#### Vektörel Grafik;

![title](vectorel1.png)

In [1]:
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from sklearn import metrics
import re
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import pandas as pd
import numpy as np
import math

In [2]:
df = pd.read_csv("BulldozersTrainAndValid.csv", low_memory=False, parse_dates=["saledate"])

In [3]:
df

,Unnamed: 0,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339995,339995,2355614,31500.0,1735417,1894,136,NaN,2001,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN
339996,339996,2355615,22500.0,1723732,4764,136,1.0,2001,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,Differential Steer,NaN,NaN
339997,339997,2355616,47000.0,1720827,1893,136,NaN,2001,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN
339998,339998,2355617,33000.0,1754587,1893,136,NaN,2002,1860.0,Low,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN


In [4]:
df.drop("Unnamed: 0", inplace=True, axis=1) # Clear useless column

In [5]:
df

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339995,2355614,31500.0,1735417,1894,136,NaN,2001,0.0,NaN,2008-10-04,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN
339996,2355615,22500.0,1723732,4764,136,1.0,2001,0.0,NaN,2008-12-16,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,Differential Steer,NaN,NaN
339997,2355616,47000.0,1720827,1893,136,NaN,2001,0.0,NaN,2007-05-10,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN
339998,2355617,33000.0,1754587,1893,136,NaN,2002,1860.0,Low,2008-02-03,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN


In [6]:
df = df.sort_values(by="saledate")

In [7]:
df["SalePrice"] = np.log(df.SalePrice)

#### Kategorik Değişkenleri Sayısal Hale Getirmek

In [8]:
df["UsageBand"]

205615     NaN
168619     NaN
28927      NaN
63166      NaN
140922     NaN
          ... 
23972     High
23971     High
23970     High
23968      Low
23967      Low
Name: UsageBand, Length: 340000, dtype: object

In [9]:
# String dtype'ı olan columları pandas category tipine çevirir
def train_cats(df):
    for n,c in df.items():
        if is_string_dtype(c):
            df[n] = c.astype("category").cat.as_ordered()

In [10]:
# train set e train_cat uygulandıktan sonra aynı category değişimlerin olsun diye validation a train'e bu uygulanır
def apply_cats(df, train):
    for n, c in df.items():
        if train[n].dtype == "category":
            df[n] = pd.Categorical(c, categories = train[n].cat.categories, ordered=True)

In [11]:
train_cats(df)

In [12]:
df["UsageBand"]

205615     NaN
168619     NaN
28927      NaN
63166      NaN
140922     NaN
          ... 
23972     High
23971     High
23970     High
23968      Low
23967      Low
Name: UsageBand, Length: 340000, dtype: category
Categories (3, object): ['High' < 'Low' < 'Medium']

In [13]:
df["UsageBand"].cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

* High > Low > Medium dizilişi biraz yanıltıcı olabilir. O yüzden ufak bir değişiklik yapalım.

In [15]:
df["UsageBand"].cat.set_categories(["High", "Medium", "Low"], ordered =True, inplace = True)

c:\Users\pc4\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arrays\categorical.py:2747: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


In [16]:
df["UsageBand"].cat.categories

Index(['High', 'Medium', 'Low'], dtype='object')

In [17]:
df["UsageBand"].loc[100:200]

100       Medium
6339      Medium
5888      Medium
136345       NaN
4141      Medium
           ...  
9086      Medium
9115      Medium
321071       NaN
299441       NaN
200       Medium
Name: UsageBand, Length: 72892, dtype: category
Categories (3, object): ['High' < 'Medium' < 'Low']

In [18]:
df["UsageBand"].cat.codes #Donusturmek icin df["UsageBand"] = df[UsageBand"].cat.codes

205615   -1
168619   -1
28927    -1
63166    -1
140922   -1
         ..
23972     0
23971     0
23970     0
23968     2
23967     2
Length: 340000, dtype: int8

* -1, 0, 2 gibi numaralandırma yanıltıcı olabilir. Onun yerine 0, 1, 3 daha kolay olacaktır.

In [22]:
def numericalize(df, col, name):
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes + 1 # ortak ağız için

In [23]:
numericalize(df, df["UsageBand"], "UsageBand")

In [24]:
df["UsageBand"]

205615    0
168619    0
28927     0
63166     0
140922    0
         ..
23972     1
23971     1
23970     1
23968     3
23967     3
Name: UsageBand, Length: 340000, dtype: int8

#### Datetime Column

In [25]:
df["saledate"]

205615   1989-01-17
168619   1989-01-31
28927    1989-01-31
63166    1989-01-31
140922   1989-01-31
            ...    
23972    2011-12-29
23971    2011-12-29
23970    2011-12-29
23968    2011-12-29
23967    2011-12-29
Name: saledate, Length: 340000, dtype: datetime64[ns]

In [26]:
df["saledate"].dt.year

205615    1989
168619    1989
28927     1989
63166     1989
140922    1989
          ... 
23972     2011
23971     2011
23970     2011
23968     2011
23967     2011
Name: saledate, Length: 340000, dtype: int64

In [27]:
def add_datepart(df, dt_name, drop=True):
    dt_column = df[dt_name]
    # column_dtype = dt_column.dtype
    attr =  ['year', 'month', 'week', 'day', 'dayofweek', 'dayofyear', 
    'is_month_end', 'is_month_start', 'is_quarter_end', 'is_quarter_start', 'is_year_end', 'is_year_start']

    for a in attr:
        df["Date" + a.capitalize()] = getattr(dt_column.dt, a) # df["saledate"].dt.year, getattr(df["saledate"].dt, "year")
    # df["Date" + 'Elapsed'] = dt_column.astype(np.int64) // 10 ** 9

    if drop:
        df.drop(dt_name, axis=1, inplace=True)

In [28]:
add_datepart(df, "saledate")

C:\Users\pc4\AppData\Local\Temp\ipykernel_8564\604729099.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df["Date" + a.capitalize()] = getattr(dt_column.dt, a) # df["saledate"].dt.year, getattr(df["saledate"].dt, "year")


In [29]:
df.head(10)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,DateWeek,DateDay,DateDayofweek,DateDayofyear,DateIs_month_end,DateIs_month_start,DateIs_quarter_end,DateIs_quarter_start,DateIs_year_end,DateIs_year_start
205615,1646770,9.159047,1126363,8434,132,18.0,1974,NaN,0,TD20,...,3,17,1,17,False,False,False,False,False,False
168619,1595295,9.680344,1291243,8298,132,18.0,1977,NaN,0,750,...,5,31,1,31,True,False,False,False,False,False
28927,1259973,8.948976,1150485,6788,132,18.0,1978,NaN,0,580,...,5,31,1,31,True,False,False,False,False,False
63166,1329619,10.221941,436674,4089,132,99.0,1987,NaN,0,D3B,...,5,31,1,31,True,False,False,False,False,False
140922,1504404,10.165852,1310407,4139,132,99.0,1975,NaN,0,D7G,...,5,31,1,31,True,False,False,False,False,False
32317,1265202,9.952278,1385055,6788,132,99.0,1984,NaN,0,580,...,5,31,1,31,True,False,False,False,False,False
98346,1415646,10.463103,1262088,3357,132,99.0,1975,NaN,0,12G,...,5,31,1,31,True,False,False,False,False,False
205752,1646995,9.259131,1044876,8443,132,99.0,1973,NaN,0,40,...,5,31,1,31,True,False,False,False,False,False
135376,1493279,11.050890,1363756,2759,132,99.0,1987,NaN,0,D5HII,...,5,31,1,31,True,False,False,False,False,False
134080,1491298,10.736397,1322436,4107,132,99.0,1987,NaN,0,D4H,...,5,31,1,31,True,False,False,False,False,False


In [30]:
for e in df.columns:
    print(e)

SalesID
SalePrice
MachineID
ModelID
datasource
auctioneerID
YearMade
MachineHoursCurrentMeter
UsageBand
fiModelDesc
fiBaseModel
fiSecondaryDesc
fiModelSeries
fiModelDescriptor
ProductSize
fiProductClassDesc
state
ProductGroup
ProductGroupDesc
Drive_System
Enclosure
Forks
Pad_Type
Ride_Control
Stick
Transmission
Turbocharged
Blade_Extension
Blade_Width
Enclosure_Type
Engine_Horsepower
Hydraulics
Pushblock
Ripper
Scarifier
Tip_Control
Tire_Size
Coupler
Coupler_System
Grouser_Tracks
Hydraulics_Flow
Track_Type
Undercarriage_Pad_Width
Stick_Length
Thumb
Pattern_Changer
Grouser_Type
Backhoe_Mounting
Blade_Type
Travel_Controls
Differential_Type
Steering_Controls
DateYear
DateMonth
DateWeek
DateDay
DateDayofweek
DateDayofyear
DateIs_month_end
DateIs_month_start
DateIs_quarter_end
DateIs_quarter_start
DateIs_year_end
DateIs_year_start


#### Feather Format

In [31]:
import os

In [32]:
df.tail()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,DateWeek,DateDay,DateDayofweek,DateDayofyear,DateIs_month_end,DateIs_month_start,DateIs_quarter_end,DateIs_quarter_start,DateIs_year_end,DateIs_year_start
23972,1222821,11.512925,1016175,16725,121,3.0,2005,9341.0,1,WA600-3,...,52,29,3,363,False,False,False,False,False,False
23971,1222820,11.512925,1010621,16725,121,3.0,2005,9946.0,1,WA600-3,...,52,29,3,363,False,False,False,False,False,False
23970,1222817,9.305651,999753,25175,121,3.0,2002,7140.0,1,570MXT,...,52,29,3,363,False,False,False,False,False,False
23968,1222813,11.532728,1038562,13316,121,3.0,2006,1764.0,3,CX290,...,52,29,3,363,False,False,False,False,False,False
23967,1222811,10.518673,1049475,13284,121,3.0,2001,3197.0,3,821C,...,52,29,3,363,False,False,False,False,False,False


In [33]:
os.makedirs('tmp', exist_ok=True) # Eğer tmp diye bir klasör var ise hata vermemesini sağlar
df.reset_index().to_feather('tmp/bulldozers_1')